In [1]:
%ls ../data

Business_Licenses.csv  fortune1000.csv
README.md              imputed_failures.csv


In [21]:
from datetime import datetime 
from pathlib import Path

import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib2tikz
from difflib import SequenceMatcher

#https://stackoverflow.com/questions/17388213/find-the-similarity-metric-between-two-strings
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

%matplotlib inline

In [3]:
data_dir = Path("../data/")
biz = pd.read_csv(data_dir/"Business_Licenses.csv", low_memory=False)
f1k = pd.read_csv(data_dir/"fortune1000.csv", low_memory=False)

In [12]:
company_names = set(f1k["Company"].apply(str).apply(str.lower))

In [18]:
sum(biz["LEGAL NAME"].apply(str).apply(str.lower).isin(company_names))/len(biz)

0.0051355103768277515

In [23]:
[_ for _ in company_names if 'walgreen' in _], set([_ for _ in biz["LEGAL NAME"].apply(str).apply(str.lower) if 'walgreen' in _])

(['walgreens boots alliance, inc'],
 {'walgreen co.', 'walgreens home care, inc.'})

In [26]:
similar('walgreens boots alliance, inc', 'walgreen co.'),\
similar('walgreens boots alliance, inc', 'walgreens home care, inc.')

(0.4878048780487805, 0.7037037037037037)

In [28]:
similar('walgreens boots alliance', 'walgreens'),\
similar('walgreens boots alliance', 'walgreens home care')

(0.5454545454545454, 0.6511627906976745)

In [39]:
similar('walgreens boots alliance', 'walgreens co.'),\
similar('walgreens boots alliance', 'walgreens home care, inc.')

(0.5945945945945946, 0.6530612244897959)

In [35]:
def name_similar_f1k(name):
    (score, match) = max(((similar(name, co), co) for co in company_names), key = lambda tup:tup[0])
    if score > 0.5: 
        return (score, match)
    return None
biz_subset = biz.iloc[0:1000]
biz_subset["matched_name"] = biz_subset["LEGAL NAME"].apply(str).apply(str.lower).apply(name_similar_f1k)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [38]:
biz_subset[["LEGAL NAME", "matched_name"]][biz_subset["matched_name"].notnull()]

,LEGAL NAME,matched_name
0,BELL OIL TERMINAL INC,"(0.6530612244897959, honeywell international i..."
1,BUCCI BIG & TALL INC.,"(0.5909090909090909, caci international inc.)"
2,"PROJECT: VISION , INC.","(0.6829268292682927, pc connection, inc.)"
3,FOLASHADE'S CLEANING SERVICE INC.,"(0.5666666666666667, total system services, inc.)"
4,WALGREEN CO.,"(0.6363636363636364, amgen inc.)"
5,"BURKS HEATING AND COOLING SOLUTIONS, LLC","(0.5789473684210527, broadridge financial solu..."
6,BELL OIL TERMINAL INC,"(0.6530612244897959, honeywell international i..."
8,BELL OIL TERMINAL INC,"(0.6530612244897959, honeywell international i..."
9,WALGREEN CO.,"(0.6363636363636364, amgen inc.)"
10,MARIA G CAHUE,"(0.5161290322580645, altria group, inc.)"


In [40]:
import string
s = "KAGIZA, INC."
s.translate(str.maketrans('', '', string.punctuation))

'KAGIZA INC'